In [2]:
import pandas as pd


In [3]:
df_sql = pd.read_csv("/content/credit_cv")
df_sql.head()

,Ind_ID,Gender,Car_owner,Property_owner,Annual_income,Income_type,Education,Marital_status,Housing_type,Phone,Email_Id,Occupation_type,Family_Members,label,Age,Years_Employed
0,5008827,M,Y,Y,180000.0,Pensioner,Higher education,Married,House / apartment,0,0,other,2,1,51.395990,0.000000
1,5009744,F,Y,N,315000.0,Commercial associate,Higher education,Married,House / apartment,1,0,other,2,1,37.117805,1.266654
2,5009746,F,Y,N,315000.0,Commercial associate,Higher education,Married,House / apartment,1,0,other,2,1,39.874878,1.266654
3,5009749,F,Y,N,202500.0,Commercial associate,Higher education,Married,House / apartment,1,0,other,2,1,37.117805,1.266654
4,5009752,F,Y,N,315000.0,Commercial associate,Higher education,Married,House / apartment,1,0,other,2,1,37.117805,1.266654


In [4]:
import duckdb
conn=duckdb.connect()

In [5]:
conn.register('df_sql',df_sql)

**1.Group the customers based on their income type and find the average of their incomes**

In [6]:

result1 = conn.execute("select Income_type, Avg(Annual_income) as Average_Income from df_sql group by Income_type;").fetchdf()
print(result1)

            Income_type  Average_Income
0             Pensioner   151512.641264
1               Working   178452.474279
2         State servant   202655.172414
3  Commercial associate   214897.808219


**2.Find the female owners of cars and property.**

In [8]:
result2 = conn.execute("select Gender,Car_owner,Property_Owner from df_sql where Gender = 'F' and Car_owner = 'Y' and Property_Owner = 'Y';").fetchdf()
result2

,Gender,Car_owner,Property_owner
0,F,Y,Y
1,F,Y,Y
2,F,Y,Y
3,F,Y,Y
4,F,Y,Y
...,...,...,...
174,F,Y,Y
175,F,Y,Y
176,F,Y,Y
177,F,Y,Y


**3.Find the male customers who are staying with their families.**

In [9]:
result3 = conn.execute("select Gender,Family_Members from df_sql where Gender = 'M' and Family_Members>0;").fetchdf()
result3

,Gender,Family_Members
0,M,2
1,M,3
2,M,3
3,M,1
4,M,3
...,...,...
562,M,2
563,M,2
564,M,3
565,M,4


**4.List the top 5 people having the highest income**

In [10]:
result4 = conn.execute("select * from df_sql order by Annual_income desc limit 5;").fetchdf()
result4

,Ind_ID,Gender,Car_owner,Property_owner,Annual_income,Income_type,Education,Marital_status,Housing_type,Phone,Email_Id,Occupation_type,Family_Members,label,Age,Years_Employed
0,5010864,M,Y,Y,380250.0,Commercial associate,Secondary / secondary special,Married,House / apartment,1,1,Core staff,3,1,49.755984,1.362461
1,5010868,M,Y,Y,380250.0,Pensioner,Secondary / secondary special,Married,House / apartment,1,1,Core staff,3,1,49.755984,1.362461
2,5010869,M,Y,Y,380250.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,1,1,Core staff,1,1,49.755984,1.362461
3,5021303,M,N,N,380250.0,Pensioner,Higher education,Married,With parents,0,1,other,3,1,24.386538,1.581047
4,5024213,F,Y,Y,380250.0,Commercial associate,Higher education,Married,House / apartment,1,0,other,2,1,42.990616,0.711697


**5.How many married people are having bad credit**

In [11]:
result5 = conn.execute("select count(*) as MP_BC from df_sql where Marital_status= 'Married' and label=1;").fetchdf()
result5

,MP_BC
0,114


**6.What is the highest education level and what is the total count**

In [12]:
result6 = conn.execute("select Education,count(*) as total_count from df_sql group by education order by total_count desc limit 1").fetchdf()
result6

,Education,total_count
0,Secondary / secondary special,1030


**7.Between married males and females who is having more bad credit?**

In [13]:
result7 = conn.execute("select Gender, Marital_status, COUNT(*) AS Bad_Credit_Count from df_sql where Marital_status = 'Married' and label = 1 group by Gender, Marital_status;").fetchdf()
result7    # label = 1 .Here 1 represents bad credits according to the given data .

,Gender,Marital_status,Bad_Credit_Count
0,F,Married,63
1,M,Married,51


Female are having more bad credit in this case